# 创建自定义转换器

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [35]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [36]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### Implement the StartingVerbExtractor class

In [54]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))
            
            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### Run program to test

In [55]:
def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y


def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline


def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


def main():
    X, y = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = model_pipeline()
    
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    display_results(y_test, y_pred)

main()

['Pfizer Animal Health to acquire Synbiotics Corporation, entering Diagnostics Sector http://bit.ly/h6inx0']
['2012 NestlÌ© CSV Prize is open for entries- $480,000 for an innovative development project.', 'Hear from the 2010 winner: http://bit.ly/jy64gj']
['New Citi-Commissioned EIU Report Projects Competitiveness of 120 of the WorldÌ¢?', '?s Major Cities in 2025 http://t.co/6735K8wUmB']
['Steven Heymsfield:  Chronic non-communicable disease is the major health problem of the day  #NINS2013']
['NestlÌ© Health Science CEO Luis Cantarell: \x89ÛÏOur stake in Accera is a strategic step forward in building up our brain health portfolio.']
['Citigroup Said to Plan Three-Year Benchmark Securities Offering:  http://t.co/Erpocz5LGR #Citigroup #BRK']
['Is your city prepared for #emergencies?', 'See how @ibmbrasil helped Rio create a unique ops center: http://t.co/Hw0lLx9VMh #FTCitiAwards']
['#Citi LinkedIn Connect Group recognized as successful network for career &amp  #finance content http://t.

['The #D2C model empowers Indonesian mothers to be entrepreneur women by selling #AQUA to household around them: they are called #AQUA Ladies']
['@News_Reviews We asked FLA to map our cocoa supply chain.', "Today we've published action plan to work on their recos http://t.co/EYDDPnOA"]
['In #Cameroon, 80 #CitiVolunteers are celebrating Global Community Day by serving a local hospital by refurbishing the pediatric ward &amp  more.']
['Can #soapoperas teach business skills?', 'See how Aprenda does this: http://t.co/kXtrvdKH8L #FTCitiAwards #progressmakers']
['Join #TSN in celebrating Bruce Voeller\x89Ûªs contributions to #science and #humanity http://t.co/84E13ZH  #LGBTPrideMonth']
['#Danone supports the @H4HInitiative: an initiative to highlight the importance of healthy #hydration http://t.co/4igbFdc8iU']
['Senior #Scientist @hgukasyan blogs about benefits of #CommunityService in San Diego FaceLift Project http://t.co/b7JH0FTP']
['US investment banks embrace riskier block trades:  http

['Pfizer colleague Kate Bird presenting at #BDI conference on Mobile Healthcare Communications http://bit.ly/hUQ2t2']
['@Stun NestlÌ© believes access to water should be a human right.', 'Please visit our Chairman\x89Ûªs water blog: http://t.co/EkadcUB5rK']
['Pfizer Global Health Fellow Tracy Sienko addressing health issues for underserved populations from Tanzania #GHF http://t.co/3vCd6CPi']
['Citigroup Sued Over $1 Billion Mortgage-Loan Securities:  http://t.co/m31lA3R9Tl #Citigroup #BRK']
['Citi Team USA Flag Tour Stops in Tucson to Celebrate Sponsorship of 2012 U.S. Olympic & Paralympic Teams http://t.co/cINSbZoR']
['&gt  20mil Americans lack access to healthy #food.', '@RevolutionFoods is building lifelong healthy eaters http://t.co/A6afGJKbj0 #progressmakers']
['Pleased to share that Pfizer\x89Ûªs Centrumå¨ Silverå¨ was used in recently published Physicians Health Study II: Learn more http://t.co/zf9sHDWn']
["Barclays Wealth's Aaron Gurwitz is urging clients to stick with US equit

['Enrobing foods can make them last longer, particularly those prone to drying out.', '#ChocolateCoveredFacts http://t.co/YU7gVqMnTZ']
["What #Nestle's Hiring Plan Says About EU's Workforce via @BloombergTV http://t.co/jCEaClB2hb"]
['With #Citi support, free program w/ @NYCBusSolutions helping entrepreneurs turn their handiwork into microbusinesses: http://t.co/TKcJ7wni18']
['NestlÌ© Nutrition Institute workshop focuses on micronutrient deficiencies http://bit.ly/gck0ks  #nutrition']
['Women who quit smoking even into their 40s/50s can still reclaim some health benefits enjoyed by nonsmokers http://t.co/5sFClAOJ8u']
['#IOM: No evidence of major safety concerns associated with adherence to the childhood immunization schedule.', 'http://t.co/lRGRHk6G']
['Well done team!', 'RT @gmitcalfe #CitiVolunteers about to start work at Spitalfields City Farm #London @spitz_cityfarm http://t.co/1vjo00qRch']
['Pfizer Global Health Fellow Albert Dunne #GHF addressing health issues for underserved popu

['Discuss patient safety and health information technology with TSN Member Vera Rulon at http://bit.ly/aCL7QB']
["We've launched Nestle Careers Facebook page!", 'LIKE and learn more about working at #Nestle http://t.co/BallRJDB1K http://t.co/s5F9tJXY3z']
['The #HealthyKidsQuest offers teachers free educational material to use with students: http://t.co/echwusSsIt @AgMuseum']
['2 Banks You Can Buy Right Now:  http://t.co/lzatdIFeRA #Citigroup #BRK']
['Pfizer appoints Mikael Dolsten President of Worldwide Research and Development: http://bit.ly/bfUhAX']
['@GCIHealth Kudos to the diabetes team!', 'Congrats!']
['To celebrate Global Community Day more than 5,000 #CitiVolunteers in #Singapore are participating in charity carnival http://t.co/XCIpc2HKfd']
['Congrats #Argentina!', 'RT @melisaech @citi #CitiVolunteers #100Years #DiaDeLaComunidad http://t.co/bZEBlJC1lA']
['Michael Porter, Professor at the Harvard Business School: more companies should work with the Creating Shared Value concept 

...

["#Leadership Tip: There's no limit to a team\x89Ûªs success if you figure out how to hire &amp  motivate the right people.", 'http://t.co/TfYZ4RWrtJ @yec']
['#HealthyBytes 5: How much water do you drink a day?', "Here's a tip for staying hydrated in the office!", 'http://t.co/YnUyk7lbUi']
['Do you know why \x89ÛÏfrugality, agility &amp  openness\x89Û\x9d is the new recipe for innovation?', 'Learn more about @JugaadAtWork !', 'http://t.co/oFeGMA1KiV']
['AAHRPP Pres.', '&amp  CEO Marjorie A. Speers says Pfizer\x89Ûªs AAHRPP accreditation raises bar, sets an example for other sponsors to follow']
['#MerckforMothers launches its U.S. program with leading maternal health organizations!', 'Learn more: http://t.co/Blw1ptBTnq']
["Tracking Bruce Berkowitz's Fairholme Portfolio - Q4 2012 Update:  http://t.co/YwrTl466 #Citigroup #BRK"]
['@2xDonations Proud to have such passionate employee volunteers!', 'http://t.co/NVgvcgiuon']
['It is estimated that nearly 1 in 8 women in the US will develop br

['"How The Most Powerful People In The World Define Healthy Living" http://t.co/xBcJOI2BNd via @HealthyLiving #wef2014']
['PFE is sponsoring a series of talks on healthy aging at @HTLGIFestival http://t.co/WQHon4eDs8 #HTLGI2013 @pfizer']
['@the_real_bob_sa Our #Nestle NAN H.A.', '1 Gold infant formula is safe.', 'http://t.co/OsLjuDg1  for more details.']
['@HealthBridgeHIO Awesome!', 'Thanks for sharing!']
['Our partnership with the IFRC &amp  Red Crescent Society of Cote D\x89ÛªIvoire focuses on investments in water, food security and emergency relief.']
['John Foreyt  Longer-term treatments have shown promise in encouraging people to maintain healthier behaviours for the long-term  #NINS2013']
['On this #HappyDay, and always, remember a healthy you is a happy you.', 'Check out these #BeWell health tips: http://t.co/wl5EpCRvTP']
['On The Doctors, Freda Lewis-Hall, MD, discusses #menopause and tests for women\x89Ûªs health after 50 http://t.co/s0JnLipI']
['@WhoopWhoop NestlÌ© believes 

['L Freixe says holding this event in Athens to show support to one of the countries most affected by this crisis - Greece #employment4youth']
['Great work team!', 'RT @JeabWara ??????', '???????', '??????', '#citivolunteers #bangkok #freedomfish ngampol_p dadathiti aa']
['Exclusive: Hedge fund Portman Square scales back, CEO steps down:  http://t.co/L8UiwCj5oh #Citigroup #BRK']
['RT WSJ Wealth Report One Third of Millionaires Use Social Media http://ow.ly/1dhxCk']
['Energy security and American jobs: President Barack Obama has a big decision to make about thi... http://t.co/YZ906ScWnn #Citigroup #BRK']
['Wealth &amp  Investment Management wins Wealth Manager of the Year award #GlobalInvestorAwards http://t.co/6z8N3xZg3b']
['Think Science Now Member Tamara Evans asks how can we all work together to avert drug shortages?', 'Read the post: http://on.pfizer.com/iiQzdK']
["@SpringboardEnt Oracle Award winner, Pfizer's Adele Gulfo, @gulfoa: At inflection points, the more challenging path wi

['Yesterday, these #HealthyKids lit up Broadway with #Nestle, @iaaforg and some sporting stars: http://t.co/YdtBj60Ofz']
['Reuters: Philanthropy outlook upbeat, but not for religious charities http://bit.ly/126ql1']
['The #Danone #Ecosystem Fund supports the Terragr\x89Ûªeau #methanisation project http://t.co/rzRLNjDRv4']
['Citigroup agrees to buy back securities from Hawaii ahead of schedule: Citigroup Inc. has agre... http://t.co/waGRsIZLE2 #Citigroup #BRK']
['Pfizer to acquire Ferrosan\x89Ûªs Consumer Healthcare Business http://on.pfizer.com/eG1wO3']
['#loreal stake in #Galderma transferred to Nestle.', '#Nestle Skin Health will be built on foundations of Galderma']
['Study could smooth lumps from powdered drinks: http://t.co/ZYHtV8AowD  #GoodFoodGoodLife http://t.co/qmwqoY3GLZ']
['3 Best Practices for Hiring (Tip: Finding the right fit is as important if not &gt  important than anything else) http://t.co/X7WE4qDaLF @yec']
['Derek White: \x89Û÷We are still learning, understanding an

['It\x89Ûªs Earth Day!', 'Help make the world a healthier place by quitting smoking.', 'Find out how at http://t.co/wwGouQrH']
['In 2011 #Danone and the IUF signed the first ww agreement on health, safety, working conditions and stress: http://t.co/2YNJiLPg #people']
['&gt; 20mil Americans lack access to healthy #food.', '@RevolutionFoods is building lifelong healthy eaters http://t.co/A6afGJKbj0 #progressmakers']
['2012 Pfizer Global Health Fellow Tracy Sienko will address health issues for underserved populations in Tanzania http://t.co/gNL8SkQN']
["Citigroup chairman says 'major work' done in shedding assets: The chairman of Citigroup Inc sa... http://t.co/PlDPfppTFe #Citigroup #BRK"]
['Exclusive: Goldman to begin fresh round of job cuts:  http://t.co/H29QNcWOft #Citigroup #BRK']
['@ASCO Thank you for a great meeting!', "We've especially enjoyed following the conversation in social media.", '#ASCO14']
['#Nestle CEO "Nutrition, health and wellness is our strongest profitable growth p